In [1]:
import psycopg2
import geopandas as gpd
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

# CONEXIÓN a la BASE DE DATOS

params = config()
conn = psycopg2.connect(**params)

zona = 'madrigal'

# CONDICIONES SINÓPTICAS

sql = '''SELECT * 
        FROM incendios_proximos_ze 
        ORDER by tot desc
        '''

incendios_cs = gpd.read_postgis(sql, conn)[:20]

tabla = incendios_cs[['idpif', 
                      'deteccion', 
                      'municipio', 
                      'paraje', 
                      'superficiearboladatotal', 
                      'superficinoarboladatotal', 
                      'tot']]

tabla.columns = ['Id PIF', 'fecha', 'municipio', 'paraje', 'sup arbolada', 'sup no arbolada', 'sup total']

condiciones_sinopticas = []

for key, incendio in incendios_cs.iterrows():
    condiciones_sinopticas.append({
        'fecha': incendio['deteccion'],
        'municipio': incendio['municipio'],
        'paraje': incendio['paraje'],
        'idcausa': incendio['idcausa'],
        'idpif': incendio['idpif'],
        'sup_arbolada': incendio['superficiearboladatotal'],
        'sup_no_arbolada': incendio['superficinoarboladatotal'],
        'sup_total': incendio['tot'],
    }
    )
    
# SEVERIDAD 

incendios = [{'zona': 'Jerte-Tornavacas', 'idpif': ['1133522', '1143015', '1143273', '574176', '1164099']}, 
             {'zona': 'Villanueva-Madrigal', 'idpif': ['1122656', '1132420', '1133464', '1163320', '1163391']}, 
             {'zona': 'Losar', 'idpif': ['1163407', '569396', '1169767', '574250', '1122672']}]

severidad = []

zonas_dict = {'jerte': 0, 'tornavacas': 0, 'madrigal': 1, 'losar': 2}

severidad = incendios[zonas_dict[zona]]
severidad['incendios'] = []

for n in severidad['idpif']:
    perimetro = gpd.read_postgis("select * from perimetros_utm where idpif = {}".format(n), conn)
    severidad['incendios'].append({'fecha': perimetro.iloc[0]['fecha'].strftime('%d/%m/%Y'), 'idpif': n})
        
# CATASTRO 

catastro = []
sql = '''
    SELECT a.*
    FROM catastro_{} a, zona_estudio_{} b
    WHERE st_intersects(a.geom, b.geom) 
    AND a.tipo = 'R'
    AND st_area(st_intersection(a.geom, b.geom)) > 50000
'''.format(zona, zona)

parcelas = gpd.read_postgis(sql, conn)
for key, parcela in parcelas.iterrows():
    catastro.append({'key': key,
                     'refcat': parcela['refcat'], 
                     'fecha': parcela['fechaalta'], 
#                      'propietario1': parcela['propietario1'], 
#                      'propietario2': parcela['propietario2'], 
#                      'contacto1': parcela['contacto1'], 
#                      'contacto2': parcela['contacto2'], 
#                      'nombre': parcela['nombre'], 
                     'sup': "{:.2f}".format(parcela['area'] / 10000.0)})

# FORESTALES
    
forestales = pd.read_sql("select forestal, contacto from forestales where zona = '{}'".format(zona), conn)

# HIDROGRAFÍA

rios = gpd.read_postgis('''select 
                row_number() over (order by a.long_km desc) as id_mapa, a.* 
                from rios a, zona_estudio_{} b 
                where st_intersects(a.geom, b.geom) 
                and a.long_km < 50'''.format(zona), conn)

# CERRAR CONEXIÓN BASE DE DATOS

conn.close()

# EXPORTAR 

capitulos = ['introduccion', 'objetivos', 'ambito', 'requisitos', 'puestos', 'planificacion', 'elementos', 'juicio']

env = Environment(loader=FileSystemLoader('static/templates'))
template =env.get_template('base.html')
output = template.render(
    zona = zona,
    severidad = severidad,
    condiciones_sinopticas = condiciones_sinopticas,
    tabla = tabla,
    catastro = catastro,
    forestales = forestales,
    capitulos = capitulos,
    rios = rios
)

with open('static/html/final.html', 'w') as f:
    f.write(output)